In [1]:
import subprocess
import sys


# List of packages to install
packages_to_install = [
    "voila",
    "ipywidgets",
    "numpy",
    "pandas",
    "matplotlib",
    "yfinance",
    "scipy",
    "cvxopt",
    "streamlit"
]

# Run pip install for all packages silently
subprocess.run(
    [sys.executable, "-m", "pip", "install"] + packages_to_install,
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)

def install_package(package):
    try:
        __import__(package)
    except ImportError:
        subprocess.run(
            [sys.executable, "-m", "pip", "install", package], 
            stdout=subprocess.DEVNULL, 
            stderr=subprocess.DEVNULL
        )

# Install required packages if not already installed
install_package("voila")
install_package("ipywidgets")
install_package( "numpy")
install_package ("yfinance")
install_package ("pandas")
install_package ("scipy")
install_package ("matplotlib")
install_package ("cvxopt")
install_package ("streamlit")


In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
import scipy
import matplotlib.pyplot as plt
from cvxopt import matrix, solvers
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import Layout, VBox, HBox

In [3]:
def read_price_data(stock_symbols, period='5y', interval='1mo'):
    """Import price data from Yahoo Finance"""
    try:
        stock_data = yf.download(stock_symbols, period=period, interval=interval)
    except Exception as e:
        print(f"Failed to download data: {e}")
        return None

    if "Adj Close" in stock_data.columns:
        prices = stock_data["Adj Close"]
        prices = prices.fillna(method="ffill")
    else:
        prices = None

    return prices

In [4]:
def capm_beta(tickers, r_f):
    """
    Calculate the expected return for a list of stocks using the CAPM formula.
    The function retrieves historical price data from Yahoo Finance. It
    calculates the monthly returns for for the S&P 500 (the market) and the
    stocks. The function computes the CAPM-beta or each stock and used this
    value to calculate the expected return.

    Parameters:
    - tickers: A list of stock ticker symbols.
    - r_f: The risk free return rate.

    Returns:
    - expect_r_list: An array containing the expected returns for each stock.
    """
    market_data = read_price_data('^GSPC', period='5y', interval='1mo')
    stock_data = read_price_data(tickers, period='5y', interval='1mo')
    r_market = np.array(market_data.pct_change().dropna())
    expect_r_list = []
    new_order = stock_data.columns.tolist()
    for stock in new_order:
        r_stock = np.array(stock_data[stock].pct_change().dropna())
        if len(r_stock) != len(r_market):
            print(f'\nAvailable data for {stock} is less than 5 years')
            print(f'Data used from the last {(len(r_stock) + 1) / 12} years \n')
            r_market_new = r_market[-len(r_stock):]
            beta = scipy.stats.linregress(r_market_new, r_stock)
        else:
            beta = scipy.stats.linregress(r_market, r_stock)
        expect_r = r_f + beta[0] * 0.07
        expect_r_list.append(expect_r)
    return np.array(expect_r_list)


def covariance_matrix(data):
    """
    Calculate the annualized covariance matrix for the given stock data.
    The function calculates daily returns for each stock and then computes
    the covariance matrix, annualized by multiplying by 252 (average number
    of trading days in a year).

    Parameters:
    - data: DataFrame containing historical daily prices of stocks.

    Returns:
    - covariance matrix: Annualized covariance matrix of the stock returns.
    """
    daily_r = data.pct_change()
    covariance_matrix = daily_r.cov() * 252
    return covariance_matrix

In [5]:
def quadratic_programming(r_desired, r_f, cov_matrix, ex_r, min_weight, max_weight):
    solvers.options['show_progress'] = False
    n = len(ex_r)

    # QP problem setup
    # 1. Objective function
    Q = matrix(np.array(cov_matrix))  # Covariance matrix
    r = matrix(np.zeros(n))  # Linear term (zero since we do not use it)

    # 2. Constraints
    # - Sum of the weights should be equal to 1
    A = matrix(np.ones(n)).T
    b = matrix(1.0)

    # - All weights are non-negative
    # - The expected portfolio return meets or exceeds the desired return.
    # - Minimum weight for specific stocks.
    if min_weight:
        G = matrix(np.vstack((-np.eye(n), -ex_r, np.eye(n))))
        h = matrix(np.hstack((-np.array(min_weight), -r_desired, np.array(max_weight))))
    else:
        G = matrix(np.vstack((-np.eye(n), -ex_r)))
        h = matrix(np.hstack(([0.0] * n, -r_desired)))

    # Solve the QP problem
    sol = solvers.qp(Q, r, G, h, A, b)

    # Extract optimal weights
    weights = np.array(sol['x']).flatten()

    # Calculate the optimal variance
    optimal_variance = np.sqrt(sol['primal objective'] * 2)
    return weights, optimal_variance

In [6]:
def calculate_weights(tickers, r_desired, r_f, min_weight, max_weight):
    if sum(min_weight) > 1:
        raise Exception("Sum of minimum weights exceeds 1")
    if len(min_weight) != len(tickers) and min_weight != []:
        raise Exception("The list of minimum weights has an invalid length.")
    if len(max_weight) != len(tickers) and max_weight != []:
        raise Exception("The list of maximum weights has an invalid length.")
    if any(a < b for a, b in zip(max_weight, min_weight)):
        raise Exception("Invalid input for the max and min weight lists.")

    combined = list(zip(tickers, min_weight, max_weight))
    sorted_combined = sorted(combined, key=lambda x: x[0])
    sorted_tickers = [item[0] for item in sorted_combined]
    sorted_min_weights = [item[1] for item in sorted_combined]
    sorted_max_weights = [item[2] for item in sorted_combined]

    data_stocks = read_price_data(tickers, '5y', '1d')
    ex_r = capm_beta(tickers, r_f)

    if all(r_desired > r for r in ex_r):
        raise Exception("The desired return rate cannot be achieved with the selected stocks.")

    cov_matrix = covariance_matrix(data_stocks)

    results, optimal = quadratic_programming(r_desired, r_f, cov_matrix, 
                                             ex_r, sorted_min_weights, sorted_max_weights)

    data = {'Expected Return': ex_r,
            'Standard Deviation': np.diagonal(cov_matrix) ** 0.5,
            'Optimal weight': results}
    df = pd.DataFrame(data, index=sorted_tickers).T

    return results, optimal, df, ex_r, sorted_tickers

In [7]:

#The function below calculates the return and volatility for randomly generated positive weights for each stock (Monte Carlo method)
def random_weights(tickers, risk_free, num, optimal, results, r_desired):
    '''
    The function first gathers the neseccary data. Then with this data the
    function calculates the expected return and the volatility for a bunch
    of random weights. We collect the expected return and the volatility for
    each random weight combination and plot these together with the optimal
    volatility in a graph.

    Parameters:
    - tickers: A list of the tickers of the stocks.
    - risk_free: The risk free return rate.
    - num: The number of different observations we want.
    - optimal: The optimal volatility for the desired return.
    '''
    data_stocks = read_price_data(tickers, '5y', '1d')
    ex_r = capm_beta(tickers, risk_free)
    cov_matrix = covariance_matrix(data_stocks)
    tickers = data_stocks.columns.tolist()

    p_return, p_vol = [], []
    for i in range(num):
        # Random vector with elements uniform on the interval [0, 1)
        # k = np.random.rand(len(tickers))
        # Normalize the elements of the vector, such that the sum is 1
        # normalized_k = k / sum(k)
        # Different way to compute random weights is by using the dirichlet distribution.
        normalized_k = np.random.dirichlet(len(tickers) * [1.], 1)[0]
        p_return.append(np.dot(ex_r, normalized_k))
        p_vol.append(np.sqrt(normalized_k.T @ cov_matrix @ normalized_k))

    plt.scatter(p_vol, p_return, c='teal', edgecolor='black', s=20, label='Random weights')
    plt.scatter(optimal, np.dot(ex_r, results), c='red', edgecolor='black', marker='X', s=150,
                label=f'Optimal weights (r_des = {r_desired})')
    plt.xlabel('Volatility (σ)', fontweight='bold')
    plt.ylabel('Return (r)', fontweight='bold')
    plt.title('Random Portfolio Simulation With Positive Weights', fontweight='bold')
    plt.legend()
    plt.grid(True)
    plt.show()

## PORTFOLIO OPTIMISER
### Please input stock tickers, your desired return and 

In [9]:
np.random.seed(37)

#Tickers

tickers_input = widgets.Text(
    value='TSLA, AMD, AVGO, AMZN, GOOG',
    description='Tickers:',
    placeholder='Enter stock tickers separated by commas',
)

tickers_input.layout = Layout(width='1000px')
tickers_input.tooltip = 'Enter stock tickers separated by commas'

desired_return_input = widgets.FloatText(
    value=0.125,
    description='Desired Return:',
)

desired_return_input.layout = Layout(width='500px')

risk_free_input = widgets.FloatText(
    value=0.038,
    description='Risk Free Rate:',  
)

risk_free_input.layout = Layout(width='500px')

min_weight_input = widgets.Text(
    value='',
    description='Min Weights:',
    placeholder='Enter minimum weights separated by commas',
)

min_weight_input.layout = Layout(width='500px')

max_weight_input = widgets.Text(
    value='',
    description='Max Weights:',
    placeholder='Enter maximum weights separated by commas',
)

max_weight_input.layout = Layout(width='500px')

style = {'description_width': 'initial'}  # This allows the description to take the necessary width.
tickers_input.style = style
desired_return_input.style = style
risk_free_input.style = style
min_weight_input.style = style
max_weight_input.style = style

tickers_input.tooltip = 'Enter stock tickers separated by commas'
desired_return_input.tooltip = 'Desired Return'
risk_free_input.tooltip = 'Risk Free Rate'
min_weight_input.tooltip = 'Enter minimum weights separated by commas'
max_weight_input.tooltip = 'Enter maximum weights separated by commas'

simulatebutton = widgets.Button(description="Simulate Portfolio")

# Grouping widgets
inputs_row1 = HBox([tickers_input])
inputs_row2 = HBox([desired_return_input, risk_free_input])
inputs_row3 = HBox([min_weight_input, max_weight_input])
simulatebutton.layout = Layout(width='100%', height='50px')

# Displaying the layout
main_layout = VBox([inputs_row1, inputs_row2,inputs_row3, simulatebutton])
display(main_layout)




output = widgets.Output()

def on_simulate_button_clicked(b):
    
     with output:
        # Clear previous output
        clear_output()
    

    # Extract user inputs
        tickers = [ticker.strip() for ticker in tickers_input.value.split(',')]
        desired_return = desired_return_input.value
        risk_free = risk_free_input.value
    
        if min_weight_input.value.strip():
            min_weight = [float(w) for w in min_weight_input.value.split(',')]
        else:
            min_weight = [0.0] * len(tickers)
    
        if max_weight_input.value.strip():
             max_weight = [float(w) for w in max_weight_input.value.split(',')]
        else:
            max_weight = [1.0] * len(tickers)
                
        if len(min_weight) != len(tickers) and len(min_weight) > 0:
            raise ValueError("The number of minimum weights must match the number of tickers.")
            
        if len(max_weight) != len(tickers) and len(max_weight) > 0:
            raise ValueError("The number of maximum weights must match the number of tickers.")
    
        num_random_observations = 20000

        weights, optimal, df, ex_r, new_tick = calculate_weights(tickers, desired_return, risk_free, min_weight, max_weight)
            
        display(df)
        display(pd.DataFrame({
            }))
            
        print(f'Sum of risky assets weights = {np.round(sum(weights), 4)}')
        print(f'Portfolio Standard Deviation = {optimal:.4%}, Portfolio Expected Return = {np.dot(ex_r, weights):.4%}')

        plt.bar(new_tick, weights, color='teal', edgecolor='black')
        plt.xlabel('Stock Ticker', fontweight='bold')
        plt.ylabel('Portfolio Weight', fontweight='bold')
        plt.title('Optimal Portfolio Weights', fontweight='bold')
        plt.show()

        random_weights(tickers, risk_free, num_random_observations, optimal, weights, desired_return)
    
simulatebutton.on_click(on_simulate_button_clicked)
display(output)

# To add:
# - risk free asset (calculate the max sharpe ratio)

Output()

[**********************80%*************          ]  4 of 5 completed

Note: you may need to restart the kernel to use updated packages.


In [10]:
import sys
print(sys.version)

3.8.10 (default, Nov 22 2023, 10:22:35) 
[GCC 9.4.0]
